In [ ]:
pip install selenium


Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install webdriver_manager

Note: you may need to restart the kernel to use updated packages.


In [1]:
import time
import random
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

import pandas as pd

In [16]:
# Set a user agent string to mimic a non-bot user
user_agent = (
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36"
)

# Configure Chrome options with the user agent
options = webdriver.ChromeOptions()
options.add_argument(f"user-agent={user_agent}")

In [ ]:
# Initialize the Chrome WebDriver with the configured options
options = Options()
# options.add_argument("--no-sandbox")
# options.add_argument('--headless=new')
# options.add_argument("--disable-gpu")

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager(
            latest_release_url='https://googlechromelabs.github.io/chrome-for-testing/last-known-good-versions-with-downloads.json',
            driver_version='116.0.5845.96').install()), options=options)

In [17]:

url = 'https://diga.bfarm.de/de/verzeichnis'
driver.get(url)

In [18]:
# Scroll through the page to load all elements
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(random.uniform(2, 4)) 

In [19]:
# Find all the elements on the page
elements = driver.find_elements("css selector", ".entity-app")

# Initialize an empty list to store the extracted data
data = []

batch_size = 10

# Set a counter to track the number of extracted records
counter = 0

In [ ]:
elements = driver.find_elements("css selector", ".entity-app")
elements_count = len(elements)

for i in range(elements_count):
    print(f"Processing element {counter + 1}")
    info = {}
    
    # Fetch the elements again in case of stale reference
    elements = driver.find_elements("css selector", ".entity-app")
    element = elements[i]
    
    # Click on the button to go to the detailed page
    button = element.find_element("css selector", ".entity-app__button")
    driver.execute_script("arguments[0].click();", button)
    
    WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.CSS_SELECTOR, ".entity-app__app-type")))

    # Re-fetch the elements on the home page
    elements = driver.find_elements("css selector", ".entity-app")

    # Wait for the detailed page to load
    info["app_name"]= driver.find_element("css selector", ".entity-app__header__name").get_attribute("innerText")
    info["app_owner"]= driver.find_element("css selector", ".entity-app__subheader").get_attribute("innerText")
    info["app_type"]= driver.find_element("css selector", ".entity-app__app-type").get_attribute("innerText")
    
    # Extract information from the detailed page
    # Extract information from the detailed page
    # After extracting the main information from the detailed page, click the "Learn More" button to open the popup
    answers = {}
    button_elements = driver.find_elements("css selector", ".entity-app__info__more > button")
    for button_element in button_elements:
        driver.execute_script("arguments[0].click();", button_element)
        time.sleep(random.uniform(2, 4))  # Random delay to mimic human-like behavior

        # Wait for the popup to load
        # Initialize a dictionary to store the answers

        # Find all question and answer elements
        qa_elements = driver.find_elements("css selector",".entity-app-answers__questions__question, .entity-app-answers__questions__answer")

        # Loop through the question and answer elements
        for i in range(0, len(qa_elements), 2):
            question = qa_elements[i].text
            answer_element = qa_elements[i + 1]

            # Get the answer text from the answer element
            answer = answer_element.text
            print(answer, question)

            # Store the answer in the appropriate field in the answers dictionary based on the question
            if "Anwendungs-URL im Apple App Store" in question:
                answers["apple_app_url"] = answer
            elif "Versionsnummer iOS-App" in question:
                print('entered')
                answers["ios_version"] = answer
            elif "Hardwarekompatibilität iOS-App" in question:
                answers["ios_hardware_compatibility"] = answer
            elif "Softwarekompatibilität iOS-App" in question:
                answers["ios_software_compatibility"] = answer
            elif "Anwendungs-URL im Google Play Store" in question:
                answers["google_play_url"] = answer
            elif "Versionsnummer Android-App" in question:
                answers["android_version"] = answer
            elif "Hardwarekompatibilität Android-App" in question:
                answers["android_hardware_compatibility"] = answer
            elif "Softwarekompatibilität Android-App" in question:
                answers["android_software_compatibility"] = answer
            elif "Patientengruppe" in question:
                answers["patientengruppe"] = answer
            elif "Geeignete Altersgruppen" in question:
                answers["geeignete_altersgruppen"] = answer
            elif "Geeignete Geschlechter" in question:
                answers["geeignete_geschlechter"] = answer
            elif "Die DiGA ist nicht anzuwenden bei folgenden Diagnosen" in question:
                answers["diga_kontraindikationen"] = answer
            elif "Weitere nicht durch Kontraindikationen abgedeckte Ausschlusskriterien" in question:
                answers["weitere_ausschlusskriterien"] = answer
            elif "Herstellerpreis (Brutto)" in question:
                answers["herstellerpreis_brutto"] = answer
            elif "Vergütungs- / Höchstbetrag (Brutto)" in question:
                answers["verguetungs_hoechstbetrag_brutto"] = answer
            elif "Mehrkosten durch den Nutzenden der DiGA" in question:
                answers["mehrkosten_nutzenden"] = answer
            elif "Erläuterung Mehrkosten" in question:
                answers["erlaeuterung_mehrkosten"] = answer
            elif "Zusatzgeräte" in question:
                answers["zusatzgeraete"] = answer
            elif "Erläuterung der Zusatzgeräte" in question:
                answers["erlaeuterung_zusatzgeraete"] = answer
            elif "Kombinationsmöglichkeiten in Bezug auf Geräte" in question:
                answers["kombinationsmoeglichkeiten_geraete"] = answer
            elif "Diese DiGA ist verfügbar in den folgenden Sprachen" in question:
                answers["verfuegbar_sprachen"] = answer



        # Close the popup
        close_button = driver.find_element("css selector", ".modal-information__close")
        driver.execute_script("arguments[0].click();", close_button)

    # Switch back to the home page
    driver.execute_script("window.scrollTo(0, 0);")
    driver.execute_script("window.history.go(-1)")
    
    # Random delay before processing the next element
    time.sleep(random.uniform(2, 4))  # Random delay to mimic human-like behavior
    
    combined_data = {**info, **answers}
    # Add the extracted data to the list
    data.append(combined_data)
    
    # Increment the counter
    counter += 1
    
    # Check if the desired number of records is reached
    if counter == 49:
        break

# Close the WebDriver
driver.quit()

# Create a DataFrame from the extracted data
df = pd.DataFrame(data)
print(df)


In [8]:
df.to_csv('output.csv', encoding='utf-8-sig', index=False)


In [9]:
df = pd.read_csv('output.csv', encoding='utf-8')


In [21]:
driver.quit()